In [12]:
import torch
from torch.utils.data import DataLoader
from ultralytics.cfg import get_cfg
from ultralytics.utils import DEFAULT_CFG, yaml_load
from ultralytics.data.utils import check_cls_dataset, check_det_dataset
from ultralytics.data import build_yolo_dataset, build_dataloader

overrides = {
    'task': 'detect',
    'data': 'data.yaml',
    'imgsz': 640,
    'workers': 4
}

cfg = get_cfg(cfg=DEFAULT_CFG, overrides=overrides)
data_info = check_det_dataset(cfg.data)

In [13]:
ds_train = build_yolo_dataset(
    cfg=cfg,
    img_path=data_info['train'],
    batch=cfg.batch,
    data=data_info,
    mode='train',
    rect=False,
    stride=32
)

ds_val = build_yolo_dataset(
    cfg,
    img_path=data_info['val'],
    batch=cfg.batch,
    data=data_info,
    mode='val',
    rect=False,
    stride=32
)

train: Scanning D:\git-work\Model-torch\yolov8\datasets\coco128\labels\train2017.cache... 126 images, 2 backgrounds, 0 corrupt: 100%|██████████| 128/128 [00:00<?, ?it/s]
val: Scanning D:\git-work\Model-torch\yolov8\datasets\coco128\labels\train2017.cache... 126 images, 2 backgrounds, 0 corrupt: 100%|██████████| 128/128 [00:00<?, ?it/s]


In [15]:
dl_train = DataLoader(
    ds_train,
    batch_size=cfg.batch,
    num_workers=cfg.workers,
    collate_fn=ds_train.collate_fn
)

dl_val = DataLoader(
    ds_val,
    batch_size=cfg.batch,
    num_workers=cfg.workers,
    collate_fn=ds_val.collate_fn,
)

In [51]:
from ultralytics.nn.tasks import DetectionModel

model = DetectionModel()
# weight = torch.load('yolov8n.pt')
# model.load(weight)


                   from  n    params  module                                       arguments                     
  0                  -1  1       464  ultralytics.nn.modules.conv.Conv             [3, 16, 3, 2]                 
  1                  -1  1      4672  ultralytics.nn.modules.conv.Conv             [16, 32, 3, 2]                
  2                  -1  1      7360  ultralytics.nn.modules.block.C2f             [32, 32, 1, True]             
  3                  -1  1     18560  ultralytics.nn.modules.conv.Conv             [32, 64, 3, 2]                
  4                  -1  2     49664  ultralytics.nn.modules.block.C2f             [64, 64, 2, True]             
  5                  -1  1     73984  ultralytics.nn.modules.conv.Conv             [64, 128, 3, 2]               
  6                  -1  2    197632  ultralytics.nn.modules.block.C2f             [128, 128, 2, True]           
  7                  -1  1    295424  ultralytics.nn.modules.conv.Conv             [128

In [52]:
model.args = cfg
model.nc = data_info['nc']
model.names = data_info['names']

In [62]:
batch = next(iter(dl_train))

In [65]:
from ultralytics.utils.loss import v8DetectionLoss

model.cuda()
loss_fn = v8DetectionLoss(model)
optimizer = torch.optim.AdamW(model.parameters(), lr=1e-4)

from tqdm import tqdm
for epoch in range(10):
    for batch in tqdm(dl_train):
        data = {
            'img': batch['img'].to(torch.float32).cuda(),
            'cls': batch['cls'].cuda(),
            'bboxes': batch['bboxes'].cuda(),
            'batch_idx': batch['batch_idx'].cuda()
        }
        # data = {k: v.cuda() for k, v in batch.items() if isinstance(v, torch.Tensor)}
        output = model(data)


 12%|█▎        | 1/8 [00:11<01:20, 11.49s/it]

(tensor(210.9568, device='cuda:0', grad_fn=<MulBackward0>), tensor([3.2546, 5.6683, 4.2620], device='cuda:0'))


 25%|██▌       | 2/8 [00:13<00:33,  5.65s/it]

(tensor(216.0154, device='cuda:0', grad_fn=<MulBackward0>), tensor([3.4327, 5.7872, 4.2811], device='cuda:0'))


 38%|███▊      | 3/8 [00:14<00:18,  3.71s/it]

...

(tensor(217.5538, device='cuda:0', grad_fn=<MulBackward0>), tensor([3.6393, 5.6924, 4.2654], device='cuda:0'))


 12%|█▎        | 1/8 [00:11<01:20, 11.50s/it]

(tensor(212.9058, device='cuda:0', grad_fn=<MulBackward0>), tensor([3.4687, 5.6187, 4.2192], device='cuda:0'))


 12%|█▎        | 1/8 [00:13<01:31, 13.06s/it]


KeyboardInterrupt: 